In [ ]:
from gnucash_business_reports.builder import GnuCash_Data_Analysis
import numpy as np
from great_tables import GT

gda = GnuCash_Data_Analysis()
gda.year = 2024

In [ ]:
gda.sanity_checker()

In [ ]:
gda.get_balance_sheet()

In [ ]:
# print(gda.get_executive_summary(include_depreciation=False))
# df = gda.get_farm_cash_transactions(include_depreciation=True)
# print(df.dtypes)
# print(df.sort_values("account_code").groupby(["account_code", "account_name"]).sum())
# print(gda.get_summary_by_account())

# flex = gda.flexible_lease_calculator()

# flex.to_csv("export/flex.csv")
# print(gda.get_corporation_value())
# print(gda.get_personal_business_expenses())

# gda.get_production().to_csv(f"export/{gda.year}-production.csv")

In [ ]:
gda.get_balance_sheet_details()

In [ ]:
grain = gda.get_grain_invoices()
open_contracts = grain[grain["Fulfilled"] == False].set_index("Invoice", drop=True)
print(open_contracts[["Crop", "Bushels", "Code"]])
open_contracts[["Crop", "Bushels", "Code"]].groupby(["Crop", "Code"]).sum()

In [ ]:
open_contracts[["Crop", "Bushels", "Code"]].groupby(["Crop", "Code"]).sum().to_html()

In [ ]:
# gda.get_all_cash_transactions().groupby(["account_desc"]).agg("post_date", np.max)
latest_tx = gda.get_all_transactions().groupby(["account_desc"]).agg(max_date=("post_date", np.max))
latest_tx

In [ ]:
grain = (
            gda.get_commodity_stock_values(["account_desc", "commodity_guid"])
        ).reset_index().set_index("account_desc")
grain["abs_qty"] = abs(grain["qty"])
grain = grain.join(latest_tx)
grain[["abs_qty", "max_date"]]

In [ ]:
gt_tbl = GT(grain[["abs_qty", "max_date"]].reset_index())
gt_tbl = gt_tbl.tab_header(
            title="Contracted Grain",
            subtitle="Harvest 2024"
            ).tab_source_note(source_note="GNUcash + elevator loads").tab_stubhead(label="landmass").fmt_date(columns="max_date", date_style="m_day_year").fmt_integer(columns="abs_qty")
gt_tbl

In [ ]:
gt_tbl.as_raw_html()